In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
import time

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(10)
driver.maximize_window()

wait = WebDriverWait(driver, 10)

action = ActionChains(driver)

driver.get(
    "https://www.youtube.com/watch?v=zA-F9EtsRJc&list=PLuMuHAJh9g_Py_PSm8gmHdlcil6CQ9QCM"
)

titles = []
playlists = driver.find_element(
    By.XPATH,
    "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[2]/div/ytd-playlist-panel-renderer/div/div[3]",
)
title_elems = playlists.find_elements(By.ID, "video-title")
# title_elems = title_elems.find_elements(By.XPATH,'')
link_imgs = playlists.find_elements(By.CSS_SELECTOR, "img")
for title in title_elems:
    title = title.text.split("l ")[-1].split(" (with.")
    title, guest = title[0], title[1].split(")")[0] if len(title) != 1 else ""
    title = title.split(" ", 1)
    loc, food = title[0], title[1] if len(title) != 1 else ""
    titles.append([loc, food, guest])

for i, link in enumerate(link_imgs):
    action.move_to_element(link).perform()
    link.click()

    time.sleep(1)
    expand_button = driver.find_element(By.ID, "expand")
    expand_button.click()
    # 조회수
    views = driver.find_element(By.XPATH, '//*[@id="info"]/span[1]')
    views = int(views.text.split()[0].replace(",",""))
    titles[i].append(views)
    # DATETIME
    date = driver.find_element(By.XPATH, '//*[@id="info"]/span[3]')
    date = date.text.split()[1:]
    for d in date:
        titles[i].append(d)

    button = driver.find_element(
        By.XPATH, '//*[@id="primary-button"]/ytd-button-renderer/yt-button-shape/button'
    )
    button.click()

    wait.until(EC.presence_of_element_located((By.ID, "secondary")))
    time.sleep(5)

    # 페이지 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 자막 세그먼트 모두 가져오기
    segments = soup.find_all(
        "div", class_="segment style-scope ytd-transcript-segment-renderer"
    )

    # 자막 시간과 텍스트 추출
    timestamps = list()
    texts = list()

    for segment in segments:
        # timestamp = segment.find("div", class_="segment-timestamp style-scope ytd-transcript-segment-renderer").get_text(strip=True)
        # timestamps.append(timestamp)
        text = segment.find(
            "yt-formatted-string",
            class_="segment-text style-scope ytd-transcript-segment-renderer",
        ).get_text(strip=True)
        texts.append(text)

    titles[i].append(texts)


driver.quit()

492590
['Sep', '10,', '2024']
["[SUNG SI KYUNG's Grab a Bite]", "It's a mecca for good restaurants. Gangbuk.", 'Inhyeon Market.', "I'm too big.", "And people recognize me since I'm on TV and stuff.", "So I don't get to visit places like here.", 'Every time I come here, I feel so excited', 'and I want to visit all the places.', 'Is this the silver pomfret street?', 'There are a lot of silver pomfret.', 'Andongjip.', "I featured 'Andongjip' in Gyeongdong Market once.", 'I heard that they sell Baechu Jeon.', 'So many.', 'I want to visit them all. I came here today because...', 'Among the juniors', 'who participated in the song', 'Nayeon said', "that she really wanted to visit this place called 'Silbi Sikdang.' Ah!", 'She said that she visited this place a couple of times and that it was great.', 'I saw the place', "on Instagram a couple of times so I've been curious about it too.", 'It looked quite unique.', 'It was very Instagrammable.', 'The prices and', 'the food seemed perfect', 'for 

KeyboardInterrupt: 

In [14]:
import pandas as pd

df = pd.DataFrame(
    titles,
    columns=["title", "food", "guest", "views", "month", "day", "year", "script"],
)
print(df)

    title      food           guest     views month   day  year  \
0    인현시장      실비식당  노래 콜라보 이벤트 당첨자  492590.0   Sep   10,  2024   
1      부산       만우장                  999862.0   Aug   30,  2024   
2      부산     해진아나고                  699967.0   Aug   26,  2024   
3      속초  화진호 이선장네                       NaN  None  None  None   
4     남영동     옛날감자전                       NaN  None  None  None   
..    ...       ...             ...       ...   ...   ...   ...   
148  신미식당       감자탕                       NaN  None  None  None   
149    화목       순대국                       NaN  None  None  None   
150    약수       순대국                       NaN  None  None  None   
151   어머니   대성집 해장국                       NaN  None  None  None   
152   대성집      도가니탕                       NaN  None  None  None   

                                                script  
0    [[SUNG SI KYUNG's Grab a Bite], It's a mecca f...  
1    [[SUNG SI KYUNG's Grab a Bite], Okay, for toda...  
2                       